In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re, requests, csv, time

In [2]:
# Check possibility to scrap webpage - run only once
#print(requests.get('https://www.youtube.com/robots.txt').text)

In [6]:
# Procedure to return HTML of the website
def getSoup(link):
    r = requests.get(link)
    r.encoding = 'UTF-8'
    if r.status_code != 200: return None
        #time.sleep(5) # slow down as per youtube 'terms of use' to human speed
    return BeautifulSoup(r.text,'lxml')

In [26]:
# Test Link to start with:
YT_link = 'https://www.youtube.com/results?search_query=tri+sestry'

# Apply procedure
YTSoup = getSoup(YT_link)

# View nesting of tags, show only the first 10001 chars
print(YTSoup.prettify()[:10000])

<!DOCTYPE html>
<html data-cast-api-enabled="true" lang="cs">
 <head>
  <style name="www-roboto">
   @font-face{font-family:'Roboto';font-style:italic;font-weight:500;src:local('Roboto Medium Italic'),local('Roboto-MediumItalic'),url(//fonts.gstatic.com/s/roboto/v18/KFOjCnqEu92Fr1Mu51S7ACc0CsE.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:italic;font-weight:400;src:local('Roboto Italic'),local('Roboto-Italic'),url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xGIzc.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:500;src:local('Roboto Medium'),local('Roboto-Medium'),url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmEU9fChc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:400;src:local('Roboto Regular'),local('Roboto-Regular'),url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu7GxP.ttf)format('truetype');}
  </style>
  <script name="www-roboto">
   if (document.fonts && document.fon

In [44]:
# 1. Views
#ViewsVideo = YTSoup.find_all('span',{'class':'view-count'})
#ViewsVideo = YTSoup.find('span',{'id':'count'})
#ViewsVideo = YTSoup.find_all('span', attrs = {'class':'style-scope.ytd-video-meta-block'})




[]
